In [1]:
#project task 1 : Musical Instruments 

In [1]:
# import libraries 
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import numpy as np
import io
from surprise import SlopeOne, NMF,NormalPredictor, CoClustering
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore,KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy
import matplotlib.pyplot as plt

In [2]:
import json
data=[]
with open('/Users/apoorvabanubakode/Downloads/meta_Musical_Instruments.json') as f:
    for l in f:
        data.append(json.loads(l.strip()))
AppliancesmetaDf=pd.DataFrame(data)

In [3]:
AppliancesmetaDf.head()

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title
0,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",NaN,0000989983,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Cricket Rubbing the spine with the wooden sti...,NaN,"[Wood percussion, Owl whistle*, Includes woode...",NaN,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,NaN,">#141,729 in Musical Instruments (See Top 100 ...",NaN,NaN,NaN,Wooden Percussion 2 Piece Set of 3 Inch Cricke...
1,"[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",NaN,0000098906,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Frog - Rubbing its spine with the wooden stic...,NaN,"[Wood percussion, Small 3 inches, Creates orig...",NaN,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$0.91,">#1,622 in Musical Instruments (See Top 100 in...",NaN,NaN,NaN,"Wooden Percussion 3 Piece Set Frog, Cricket an..."
2,NaN,NaN,0041291905,Hal Leonard,"[Musical Instruments, Instrument Accessories, ...","May 10, 2011",[Vivaldi's famous set of four violin concertos...,NaN,"[., ., .]",NaN,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$62.93,">#330,653 in Musical Instruments (See Top 100 ...",NaN,NaN,NaN,Hal Leonard Vivaldi Four Seasons for Piano (Or...
3,"[0486266842, 0793507669, 0393008789, 142341280...",NaN,0060015500,Boosey &amp; Hawkes,NaN,"May 23, 2007","[The Turn of the Screw (op. 54) vocal score, p...",NaN,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$107.79,"[>#86,354 in Musical Instruments (See Top 100 ...",NaN,NaN,NaN,The Turn of the Screw (vocal score)
4,NaN,NaN,0193757710,NaN,NaN,"February 8, 2013",NaN,NaN,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,NaN,"[>#482,025 in Musical Instruments (See Top 100...",NaN,NaN,NaN,Suite for Organ (including the Trumpet Volunta...


In [4]:
#AppliancesmetaDf_cleaned.head()
df3 = AppliancesmetaDf.fillna('')
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
print(len(df4))
print(len(df5))

1907
118493


In [5]:
AppliancesmetaDf_cleaned=df5.copy()

In [6]:
dataAppliances=[]
with open('/Users/apoorvabanubakode/Downloads/Musical_Instruments.json') as f:
    for l in f:
        dataAppliances.append(json.loads(l.strip()))

AppliancesDf=pd.DataFrame(dataAppliances)


In [7]:
AppliancesDf.shape

(1512530, 12)

In [8]:
AppliancesDf.head()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,0470536454,NaN,5.0,Crocheting for Dummies by Karen Manthey & Susa...,"08 9, 2004",AXHY24HWOF184,Bendy,{'Format:': ' Paperback'},Terrific Book for Learning the Art of Crochet,1092009600,False,90
1,0470536454,NaN,4.0,Very helpful...,"04 6, 2017",A29OWR79AM796H,Amazon Customer,{'Format:': ' Hardcover'},Four Stars,1491436800,True,2
2,0470536454,NaN,5.0,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,"03 14, 2017",AUPWU27A7X5F6,Amazon Customer,{'Format:': ' Paperback'},Five Stars,1489449600,True,NaN
3,0470536454,NaN,4.0,My girlfriend use quite often,"02 14, 2017",A1N69A47D4JO6K,Christopher Burnett,{'Format:': ' Paperback'},Four Stars,1487030400,True,NaN
4,0470536454,NaN,5.0,Arrived as described. Very happy.,"01 29, 2017",AHTIQUMVCGBFJ,Amazon Customer,{'Format:': ' Paperback'},Very happy.,1485648000,True,NaN


In [26]:
subset_ratings=AppliancesDf[['asin','reviewTime','reviewerID','verified','overall','reviewText']]
subset_ratings.shape

(1512530, 6)

In [28]:
#verified review only
subset_ratings=subset_ratings[subset_ratings.verified==True]
subset_ratings['reviewDate']=pd.to_datetime(subset_ratings['reviewTime'])
subset_ratings.head()

,asin,reviewTime,reviewerID,verified,overall,reviewText,reviewDate
1,0470536454,"04 6, 2017",A29OWR79AM796H,True,4.0,Very helpful...,2017-04-06
2,0470536454,"03 14, 2017",AUPWU27A7X5F6,True,5.0,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,2017-03-14
3,0470536454,"02 14, 2017",A1N69A47D4JO6K,True,4.0,My girlfriend use quite often,2017-02-14
4,0470536454,"01 29, 2017",AHTIQUMVCGBFJ,True,5.0,Arrived as described. Very happy.,2017-01-29
5,0470536454,"01 4, 2017",A1J8LQ7HVLR9GU,True,5.0,Love the Dummies Series. Never fails.,2017-01-04


In [29]:
#lets see how much data in 2016 onwards :
subset_ratings[subset_ratings.reviewDate>'2017-01-01'].shape

(370108, 7)

In [30]:
# lets use recent reviews : post 2016
subset_ratings=subset_ratings[subset_ratings.reviewDate>'2018-01-01']

In [31]:
#check duplicate values and drop them (mostly reviews are updated so keep last active):
checkdf=subset_ratings[subset_ratings.duplicated(subset=['asin','reviewerID','reviewDate'],keep=False)].sort_values(['asin','reviewerID','reviewDate'])
checkdf.shape

subset_ratings=subset_ratings.drop_duplicates(subset=['asin','reviewerID','reviewDate'],keep='last')
subset_ratings.shape

(366645, 7)

In [32]:
subset_ratings=subset_ratings[['reviewerID','asin','overall']]
#currently for svd /knn models only consider product id ; reviewer id and ratings
#later when top 10 findings then select according to dates and categoty and other details of the asin id 

In [33]:
no_of_products_reviewed=subset_ratings.asin.value_counts()
no_of_products_reviewed.describe([.95,.97,.99])
# we can take products who have more than 10 reviews to start with ; but lets try iwith actual given products

count    48765.000000
mean         7.518610
std         31.184289
min          1.000000
50%          2.000000
95%         26.000000
97%         41.000000
99%        103.000000
max       1348.000000
Name: asin, dtype: float64

In [34]:
len(no_of_products_reviewed) # unique product count is 23439

48765

In [35]:
review_per_user_counts=subset_ratings.reviewerID.value_counts()
review_per_user_counts.describe([.95,.97,.99]) # unique user count is 300070

count    258896.000000
mean          1.416186
std           1.373761
min           1.000000
50%           1.000000
95%           3.000000
97%           4.000000
99%           7.000000
max          76.000000
Name: reviewerID, dtype: float64

In [37]:
atemp=subset_ratings.reviewerID.value_counts()
atemp[atemp>=3].shape

(20796,)

In [38]:
#lets see how many users we have if we consider more than 3 reviews per user 

#272798 reviewer id just has one review 
#22284 reviewer id just has two review 
#3646 reviewer id just has three review 
filtered_users=review_per_user_counts[review_per_user_counts>=3].reset_index()

In [39]:
filtered_users=filtered_users['index'].tolist()

In [40]:
subset_backup=subset_ratings.copy()

In [41]:
#filtered_users based on reviews>5
subset_ratings=subset_ratings[subset_ratings.reviewerID.isin(filtered_users)]

In [42]:
subset_ratings.shape

(97439, 3)

In [44]:
grocery=subset_ratings

In [45]:
'''grocery=pd.read_csv('processed_grocery.csv')
grocery.head()'''

"grocery=pd.read_csv('processed_grocery.csv')\ngrocery.head()"

In [46]:
grocery=grocery[['reviewerID','asin','overall']]
#cast into subset ratings
subset_ratings=grocery

In [130]:
val=subset_ratings.reviewerID.value_counts()
val=val.reset_index()
owners_10ormore=val[val.reviewerID>=10]
owners_10ormore=owners_10ormore.drop_duplicates(subset=['index'])
owners_10ormore=list(owners_10ormore['index'])
#owners_10ormore

In [131]:
len(owners_10ormore)

1183

In [49]:
subset_ratings=subset_ratings[subset_ratings.reviewerID.isin(owners_10ormore)]
subset_ratings.shape
#earlier shape -(2454435, 3)

(17268, 3)

In [132]:
#cut 30% data of users for hidden tests 
subset_ratings_train=pd.DataFrame()
subset_ratings_test=pd.DataFrame()
for owner in owners_10ormore[:]:
    ownersubset=subset_ratings[subset_ratings.reviewerID==owner]
    splitpoint=int(ownersubset.shape[0]*0.8)
    
    subset_ratings_train=pd.concat([subset_ratings_train,ownersubset.iloc[:splitpoint,:]],axis=0)
    subset_ratings_test=pd.concat([subset_ratings_test,ownersubset.iloc[splitpoint:,:]],axis=0)
    
print(subset_ratings_train.shape)
print(subset_ratings_test.shape)
    

(13393, 3)
(3875, 3)


In [51]:
subset_ratings_test.head()

,reviewerID,asin,overall
1271978,A9SINTLKCQK90,B006CQ8PGC,5.0
1272111,A9SINTLKCQK90,B006DFJ4HQ,5.0
1276267,A9SINTLKCQK90,B006W2SFPW,4.0
1277334,A9SINTLKCQK90,B006YV31JG,2.0
1294039,A9SINTLKCQK90,B008M0B10I,3.0


In [52]:
subset_ratings_train.head()

,reviewerID,asin,overall
105269,A9SINTLKCQK90,B0002F4UYU,5.0
108434,A9SINTLKCQK90,B0002F52U6,3.0
120295,A9SINTLKCQK90,B0002F7KG0,5.0
238021,A9SINTLKCQK90,B000DZXTFK,3.0
421106,A9SINTLKCQK90,B001MW9TF2,2.0


In [68]:
subset_ratings_train[subset_ratings_train['reviewerID']=='A2C9XE9I8RSKNX'].shape

(176, 3)

In [69]:
subset_ratings_test[subset_ratings_test['reviewerID']=='A2C9XE9I8RSKNX'].shape

(44, 3)

In [53]:
#subset_ratings_train.to_csv('subset_ratings_train1.csv')
#subset_ratings_test.to_csv('subset_ratings_test1.csv')

In [54]:
subset_ratings_train=pd.read_csv('subset_ratings_train1.csv')
subset_ratings_train=subset_ratings_train[['reviewerID','asin','overall']]
subset_ratings_train.head()

,reviewerID,asin,overall
0,A9SINTLKCQK90,B0002F4UYU,5.0
1,A9SINTLKCQK90,B0002F52U6,3.0
2,A9SINTLKCQK90,B0002F7KG0,5.0
3,A9SINTLKCQK90,B000DZXTFK,3.0
4,A9SINTLKCQK90,B001MW9TF2,2.0


In [55]:
subset_ratings_train['asin'].nunique()

6146

In [56]:
a=subset_ratings_train[['asin']]
#print(a.shape)
a=a.drop_duplicates()
#print(a.shape)
all_items_train=list(a.loc[:,'asin'])
#print(all_items_train)
ar=subset_ratings_train[['reviewerID']]
print(ar.shape)
ar=ar.drop_duplicates()
print(ar.shape)
all_users_train=list(ar.loc[:,'reviewerID'])
#print(all_users_train)

(13393, 1)
(1183, 1)


In [57]:
import random

#choose 30 most frequent buyers
maxrevs=subset_ratings_train.reviewerID.value_counts().reset_index()
maxrevs_id=list(maxrevs.iloc[:30,0])
maxrevs_id
sampling_users = maxrevs_id

In [59]:
sampled_itemdf=pd.DataFrame(all_items_train,columns=['asin'])
sampled_itemdf.head()
sampling_usersdf=pd.DataFrame(sampling_users,columns=['reviewerID'])
sampling_usersdf.head()

,reviewerID
0,A9SINTLKCQK90
1,A2U53XJZLPQWX2
2,A1MHPGANXXGLC8
3,A3YVD62FSVIUJ
4,AO21QAFMIK124


In [60]:
len(sampling_users)

30

In [61]:
import itertools
result= list(itertools.product( sampling_users,all_items_train))

In [62]:
resultdf=pd.DataFrame(result,columns=['reviewerID','asin'])
resultdf.head()

,reviewerID,asin
0,A9SINTLKCQK90,B0002F4UYU
1,A9SINTLKCQK90,B0002F52U6
2,A9SINTLKCQK90,B0002F7KG0
3,A9SINTLKCQK90,B000DZXTFK
4,A9SINTLKCQK90,B001MW9TF2


In [63]:
resultdf.shape
#this is test set 
resultdf['overall']=0
resultdf.head()

,reviewerID,asin,overall
0,A9SINTLKCQK90,B0002F4UYU,0
1,A9SINTLKCQK90,B0002F52U6,0
2,A9SINTLKCQK90,B0002F7KG0,0
3,A9SINTLKCQK90,B000DZXTFK,0
4,A9SINTLKCQK90,B001MW9TF2,0


In [64]:
resultdf.shape

(184380, 3)

In [65]:
#create svd from subset rating :
reader = Reader(rating_scale=(1,5)) 
data=Dataset.load_from_df(subset_ratings_train,reader)

In [66]:
#trainset, testset = train_test_split(data, test_size=.25)
trainset = data.build_full_trainset()

In [67]:
#create concat df of test and antitestset.
#trainset = trainset.build_full_trainset()
#testset = trainset.build_anti_testset()
testset=resultdf.copy()
testset= Dataset.load_from_df(testset, reader=reader)
testset = testset.build_full_trainset().build_testset()

In [83]:
'''trainset = data.build_full_trainset()
#algo.fit(trainset)
testset = trainset.build_anti_testset()'''

'trainset = data.build_full_trainset()\n#algo.fit(trainset)\ntestset = trainset.build_anti_testset()'

In [68]:
# lets try algorithms without any data reduction and see time and test rmse
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD()]:
    # Perform cross validation
    print("algorithm",algorithm)
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 


algorithm <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x118385160>


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.897276,0.493926,0.025379


In [69]:
#create a build training set ;
#create a testing set - missing ratings 

#trainingset=data.build_full_trainset()
#testset = trainingset.build_anti_testset()

In [70]:
algo = SVD(n_factors=100)
algo.fit(trainset)

In [71]:
predictions = algo.test(testset)
#accuracy.rmse(predictions, verbose=False)

In [72]:
len(predictions)

184380

In [133]:
predictions[:10]

[Prediction(uid='A9SINTLKCQK90', iid='B0002F4UYU', r_ui=0.0, est=4.426739238330217, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B0002F52U6', r_ui=0.0, est=3.5077037087912775, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B0002F7KG0', r_ui=0.0, est=4.637082736475524, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B000DZXTFK', r_ui=0.0, est=3.7840372979147685, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B001MW9TF2', r_ui=0.0, est=3.6114738657965146, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B003ZHP8IY', r_ui=0.0, est=4.458743856307638, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B00F9DQXSG', r_ui=0.0, est=4.348895016826446, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid='B00HEXV80M', r_ui=0.0, est=4.602451359780288, details={'was_impossible': False}),
 Prediction(uid='A9SINTLKCQK90', iid=

In [74]:
from collections import defaultdict
def get_top_n(predictions, n=10):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [75]:
top_n = get_top_n(predictions, n=10)

In [136]:
#top_n

In [137]:
L = [(k, *t) for k, v in top_n.items() for t in v]

recommended_df = pd.DataFrame(L, columns=['reviewerID','item','predicted rating'])
print (recommended_df.head())

      reviewerID        item  predicted rating
0  A9SINTLKCQK90  B0000AQRST          4.794425
1  A9SINTLKCQK90  B01EJ9KI5O          4.748404
2  A9SINTLKCQK90  B0002E1J3Q          4.713863
3  A9SINTLKCQK90  B00TV90MT0          4.686630
4  A9SINTLKCQK90  B01GMW448S          4.681089


In [78]:
#itemdf= pd.DataFrame(itemslist, columns =['asin', 'overall'])
#unique dict of item and their titles :
itemdict=AppliancesmetaDf_cleaned[['asin','title']]
itemdict=itemdict.drop_duplicates()
itemdict=itemdict.set_index('asin').T
itemdict=itemdict.to_dict('list')
#itemdict['1906487049']
catdict=AppliancesmetaDf_cleaned[['asin','main_cat']]
catdict=catdict.drop_duplicates()
catdict=catdict.set_index('asin').T
catdict=catdict.to_dict('list')
#catdict['1906487049']

In [79]:
recommended_df['Item Name']=recommended_df['item'].map(itemdict)
recommended_df['Category Name']=recommended_df['item'].map(catdict)
recommended_df.head()

,reviewerID,item,predicted rating,Item Name,Category Name
0,A9SINTLKCQK90,B0000AQRST,4.794425,[Shure SM57-LC Cardioid Dynamic Microphone],[Musical Instruments]
1,A9SINTLKCQK90,B01EJ9KI5O,4.748404,[Rinastore 6-String Acoustic & Electric Guitar...,[Musical Instruments]
2,A9SINTLKCQK90,B0002E1J3Q,4.713863,"[Ernie Ball Not Even Slinky Nickel Wound Set, ...",[Musical Instruments]
3,A9SINTLKCQK90,B00TV90MT0,4.686630,[Music Nomad MN206 Cradle Cube String Instrume...,[Musical Instruments]
4,A9SINTLKCQK90,B01GMW448S,4.681089,[Tibetan Singing Bowl Set By YAK THERAPY- Chak...,[Musical Instruments]


In [80]:
def pred_for_user(userid):
    temp=recommended_df[recommended_df.reviewerID==userid]
    return temp

In [81]:
'''userid='A11U6KGWNEIEKB'
suggested=pred_for_user(userid)
print(suggested['Item Name'],suggested['Category Name'],suggested['predicted rating'])'''

"userid='A11U6KGWNEIEKB'\nsuggested=pred_for_user(userid)\nprint(suggested['Item Name'],suggested['Category Name'],suggested['predicted rating'])"

In [82]:
def product_hist_for_user(userid):
    products_history=subset_backup[subset_backup.reviewerID==userid]
    products_history=products_history[['asin','overall']]
    print(products_history.shape)
    products_history=products_history.drop_duplicates(subset=['asin','overall'])
    print(products_history.shape)
    products_history=pd.DataFrame(products_history,columns=['asin','overall'])
    products_history['Item Name']=products_history['asin'].map(itemdict)
    products_history['main_cat']=products_history['asin'].map(catdict)
    return products_history

In [84]:
#trainingdf.head()

In [85]:
def product_hist_for_user_training(userid):
    products_history=trainset[subset_backup.reviewerID==userid]
    products_history=products_history[['asin','overall']]
    print(products_history.shape)
    products_history=products_history.drop_duplicates(subset=['asin','overall'])
    print(products_history.shape)
    products_history=pd.DataFrame(products_history,columns=['asin','overall'])
    products_history['Item Name']=products_history['asin'].map(itemdict)
    products_history['main_cat']=products_history['asin'].map(catdict)
    return products_history

In [86]:
subset_backup=subset_ratings_train.copy()

In [87]:
subset_ratings_test=pd.read_csv('subset_ratings_test1.csv')
subset_ratings_test=subset_ratings_test.iloc[:,1:]
subset_ratings_test.head(5)

,reviewerID,asin,overall
0,A9SINTLKCQK90,B006CQ8PGC,5.0
1,A9SINTLKCQK90,B006DFJ4HQ,5.0
2,A9SINTLKCQK90,B006W2SFPW,4.0
3,A9SINTLKCQK90,B006YV31JG,2.0
4,A9SINTLKCQK90,B008M0B10I,3.0


In [125]:
'''recommended_df.head()
recommended_df.to_csv('Recommended_Musical_Instruments.csv')'''

In [91]:
recommended_df.reviewerID.unique()

array(['A9SINTLKCQK90', 'A2U53XJZLPQWX2', 'A1MHPGANXXGLC8',
       'A3YVD62FSVIUJ', 'AO21QAFMIK124', 'A396ELNTQDFYPO',
       'A17Y6FE1XWX9MG', 'A1SXYKNFEGZMCG', 'AJORX8PDEWL7W',
       'A1ZRMGNB5NZFQX', 'A3EATLL9FW7DJL', 'AQDUE9PHA7QRP',
       'ASCOQL6HNN0OE', 'A67OJZLHBBUQ9', 'A2UZ219VYXN78J',
       'A2MZ8IKTPCMKPT', 'A3HNKTVQO3B037', 'A59G70843Q9AI',
       'A182VFGL9YR6SZ', 'AGHMAMHIW4CDQ', 'A22XP8CF9U631W',
       'A265SU6RW4ZAPY', 'A2YUDQKESBIEEU', 'A2VDKEI5V937X4',
       'A215GPS2IFRI7G', 'A2D6IKTQ6ET3MC', 'AKD1S6FHXAQJB',
       'A15LAJVR0RX15T', 'A7BTFA87UO4TZ', 'A15UON6342H4YU'], dtype=object)

In [123]:
subset_ratings_test.head()

,reviewerID,asin,overall
0,A9SINTLKCQK90,B006CQ8PGC,5.0
1,A9SINTLKCQK90,B006DFJ4HQ,5.0
2,A9SINTLKCQK90,B006W2SFPW,4.0
3,A9SINTLKCQK90,B006YV31JG,2.0
4,A9SINTLKCQK90,B008M0B10I,3.0


In [128]:
'''copy_items=subset_ratings_test.copy()
copy_items['name']=copy_items.asin.map(itemdict)
copy_items.head()
copy_items.name.isnull().sum()'''

22

In [129]:
#copy_items.to_csv('Bought_Musical_Instruments.csv')

In [121]:
userid='A2D6IKTQ6ET3MC'
subset_user=subset_ratings_test[subset_ratings_test.reviewerID==userid]
items=subset_user.asin.unique()
#items is actual bought list 
print(len(items))
items=pd.DataFrame(items,columns=['itemsact'])
items['name']=items.itemsact.map(itemdict)
print(items)

7
     itemsact                                               name
0  B00O6ZM7YM                      [MAPEX Cymbal Stand (H200RB)]
1  B00OY670QS  [Tascam DR-10X Plug-On Linear PCM Digital Reco...
2  B00U5T6JNS  [IsoAcoustics Aperta Isolation Stands - Black ...
3  B00UBD137K  [Goedrum GBD18 18" Electric Kick Drum or Elect...
4  B00W54Q6NY      [Goedrum T-Clamp for 1 1/2 inch (1.5") Poles]
5  B00XWL2VVK  [AC Power Filter Power Conditioner - WAudio AC...
6  B01C8O2RF6                     [ddrum QSDBDP Bass Drum Pedal]


In [122]:
suggested=pred_for_user(userid)
print(suggested['Item Name'],suggested['Category Name'],suggested['predicted rating'])

250    [Kirlin Cable IWB-202BFGL-20/RO -20 feet- Stra...
251    [Dunlop Tortex Sharp Guitar Picks 1.14mm Purpl...
252    [ChromaCast CC-MINIGS Universal Folding Guitar...
253    [D'Addario EXL120+ Nickel Wound Electric Guita...
254                            [Fender Pickguard Screws]
255    [Electro Harmonix Stereo Talking Machine w/2 6...
256    [D'Addario EXL120-7 Nickel Wound 7-String Elec...
257          [JJ Electronics Amplifier Tube (T-EL84-JJ)]
258    [20 Feet (10 White/10 Black) Gavitt Cloth-cove...
259                      [DrumDial Precision Drum Tuner]
Name: Item Name, dtype: object 250    [Musical Instruments]
251    [Musical Instruments]
252    [Musical Instruments]
253    [Musical Instruments]
254    [Musical Instruments]
255    [Musical Instruments]
256    [Musical Instruments]
257    [Musical Instruments]
258    [Musical Instruments]
259    [Musical Instruments]
Name: Category Name, dtype: object 250    5.000000
251    5.000000
252    4.983107
253    4.967541
254    4.9

In [94]:
#get actual test holdout items for the user 
userid='A9SINTLKCQK90'
subset_user=subset_ratings_test[subset_ratings_test.reviewerID==userid]
items=subset_user.asin.unique()
#items is actual bought list 
print(len(items))
items=pd.DataFrame(items,columns=['itemsact'])
items['name']=items.itemsact.map(itemdict)
print(items)

16
      itemsact                                               name
0   B006CQ8PGC                 [Classic Mini Djembe Drum 4" X 8"]
1   B006DFJ4HQ  [Axatse African Shaker - Maraca, Rumba Shaker ...
2   B006W2SFPW  [Meinl Percussion HDJ4-L Nile Series Headliner...
3   B006YV31JG              [Cannon UPC9 9 1/2-Inch Mega Cowbell]
4   B008M0B10I  [X8 Drums & Percussion X8-COVER-KENTE-3L Kente...
5   B00BF7XD2Y  [Meinl Percussion FJS1A-BK Aluminum Finger Jin...
6   B00F5DRWCG       [Tycoon Percussion TGO-L Agogo Bell - Large]
7   B00G5LDS9I  [Latin Percussion LP601NY-CMW City Series Carv...
8   B00GB0P094  [Latin Percussion LP243B Super Guiro Scraper -...
9   B00HJOTFI8   [Latin Percussion LP446-S Session Shaker, Small]
10  B00KHBPZGG  [DharmaObjects Tibetan Palm Size Solar Plexus ...
11  B00OJQ9TJE           [ChromaCast CC-DS-CONGA Conga Drum Sack]
12  B00Q2G2NTC  [Meinl Percussion Precision Cajon Brush with S...
13  B017LW0NCY  [Thumb Piano Mbira Music Story Telling Instrum...
14  B01

In [95]:
suggested=pred_for_user(userid)
print(suggested['Item Name'],suggested['Category Name'],suggested['predicted rating'])

0          [Shure SM57-LC Cardioid Dynamic Microphone]
1    [Rinastore 6-String Acoustic & Electric Guitar...
2    [Ernie Ball Not Even Slinky Nickel Wound Set, ...
3    [Music Nomad MN206 Cradle Cube String Instrume...
4    [Tibetan Singing Bowl Set By YAK THERAPY- Chak...
5    [Rotosound RS200 Top Tape Monel Flatwound Elec...
6    [Neewer NW(B-3) 6 inch Studio Microphone Mic R...
7    [60 Pack - Clear Empty Refillable Plastic Pot ...
8      [Fulltone OCD Obsessive Compulsive Drive Pedal]
9            [Hosa Technology GXR134 Signal Converter]
Name: Item Name, dtype: object 0    [Musical Instruments]
1    [Musical Instruments]
2    [Musical Instruments]
3    [Musical Instruments]
4    [Musical Instruments]
5    [Musical Instruments]
6    [Musical Instruments]
7             [All Beauty]
8    [Musical Instruments]
9    [Musical Instruments]
Name: Category Name, dtype: object 0    4.794425
1    4.748404
2    4.713863
3    4.686630
4    4.681089
5    4.666022
6    4.652939
7    4.651521
8 

In [96]:
#get actual test holdout items for the user 
userid='A9SINTLKCQK90'
subset_user=subset_ratings_test[subset_ratings_test.reviewerID==userid]
items=subset_user.asin.unique()
#items is actual bought list 
print(len(items))

16


In [97]:
items=pd.DataFrame(items,columns=['itemsact'])
items['name']=items.itemsact.map(itemdict)
print(items)

      itemsact                                               name
0   B006CQ8PGC                 [Classic Mini Djembe Drum 4" X 8"]
1   B006DFJ4HQ  [Axatse African Shaker - Maraca, Rumba Shaker ...
2   B006W2SFPW  [Meinl Percussion HDJ4-L Nile Series Headliner...
3   B006YV31JG              [Cannon UPC9 9 1/2-Inch Mega Cowbell]
4   B008M0B10I  [X8 Drums & Percussion X8-COVER-KENTE-3L Kente...
5   B00BF7XD2Y  [Meinl Percussion FJS1A-BK Aluminum Finger Jin...
6   B00F5DRWCG       [Tycoon Percussion TGO-L Agogo Bell - Large]
7   B00G5LDS9I  [Latin Percussion LP601NY-CMW City Series Carv...
8   B00GB0P094  [Latin Percussion LP243B Super Guiro Scraper -...
9   B00HJOTFI8   [Latin Percussion LP446-S Session Shaker, Small]
10  B00KHBPZGG  [DharmaObjects Tibetan Palm Size Solar Plexus ...
11  B00OJQ9TJE           [ChromaCast CC-DS-CONGA Conga Drum Sack]
12  B00Q2G2NTC  [Meinl Percussion Precision Cajon Brush with S...
13  B017LW0NCY  [Thumb Piano Mbira Music Story Telling Instrum...
14  B019M3

In [133]:
items.to_csv('user_A1K82R24ROO2I7_test_actual_items.csv')

In [98]:
suggested=pred_for_user(userid)
print(suggested['Item Name'],suggested['Category Name'],suggested['predicted rating'])

0          [Shure SM57-LC Cardioid Dynamic Microphone]
1    [Rinastore 6-String Acoustic & Electric Guitar...
2    [Ernie Ball Not Even Slinky Nickel Wound Set, ...
3    [Music Nomad MN206 Cradle Cube String Instrume...
4    [Tibetan Singing Bowl Set By YAK THERAPY- Chak...
5    [Rotosound RS200 Top Tape Monel Flatwound Elec...
6    [Neewer NW(B-3) 6 inch Studio Microphone Mic R...
7    [60 Pack - Clear Empty Refillable Plastic Pot ...
8      [Fulltone OCD Obsessive Compulsive Drive Pedal]
9            [Hosa Technology GXR134 Signal Converter]
Name: Item Name, dtype: object 0    [Musical Instruments]
1    [Musical Instruments]
2    [Musical Instruments]
3    [Musical Instruments]
4    [Musical Instruments]
5    [Musical Instruments]
6    [Musical Instruments]
7             [All Beauty]
8    [Musical Instruments]
9    [Musical Instruments]
Name: Category Name, dtype: object 0    4.794425
1    4.748404
2    4.713863
3    4.686630
4    4.681089
5    4.666022
6    4.652939
7    4.651521
8 

In [106]:
len(recommended_df.reviewerID.unique())

30

In [138]:
#suggested.to_csv('suggested_A1K82R24ROO2I7_items_50_exhaustivelist.csv')

In [100]:
alreadypurchased=subset_ratings_train[subset_ratings_train.reviewerID=='A9SINTLKCQK90']
alreadypurchased['name']=alreadypurchased['asin'].map(itemdict)
alreadypurchased.head()

/Users/apoorvabanubakode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,reviewerID,asin,overall,name
0,A9SINTLKCQK90,B0002F4UYU,5.0,"[Rhythm Tech Cabasa, Chrome (RT8000)]"
1,A9SINTLKCQK90,B0002F52U6,3.0,"[CP378 8"" Wood Tambourine, Headed, Single Row ..."
2,A9SINTLKCQK90,B0002F7KG0,5.0,"[Remo HD-8512-00 Fiberskyn Frame Drum, 12""]"
3,A9SINTLKCQK90,B000DZXTFK,3.0,"[Bodhran Economy Case, 16"" x 4.5""]"
4,A9SINTLKCQK90,B001MW9TF2,2.0,NaN


In [143]:
#alreadypurchased.to_csv('alreadypurchased_A1K82R24ROO2I7_items.csv')

In [105]:
unique_items_suggested=suggested.item.unique()

In [138]:
#items

In [139]:
#unique_items_suggested

In [71]:
#After looking at the recommendations manually, we saw that though he exact brands did not overlap in test holdout data and 